## BUAD 313 -- Session 7/8 (Modeling/Solving Systematically) -- Factory Planning LP

In [ ]:
import numpy as np
from gurobipy import Model, GRB, quicksum

In Session 7, we introduced `Step 0` in optimization model formulation, which is to identify the structure of the input data. We're going to start by defining index sets, and then loading in some data by reading it in from a file and manipulating it.

### Scalar Data in the Case

In [ ]:
#scalar data from the case, just written by hand
max_storage = 100
end_stock = 50
hours_per_mach = 384
storage_cost = 0.75


### Defining Index Sets

In [ ]:
#define a list of products labeled A through G
products = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

#define a list of integers from 1 to 12, representing the months of the year
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

#define a list of machines types: grinding, vertical drilling, horizontal, boring, and planing, abbreviated by first letter (capitalized)
machines = ['G', 'V', 'H', 'B', 'P']

#define a list of tasks: produce, sell, and store 
tasks = ['Produce', 'Sell', 'Store']

### Loading Indexed Data

In [ ]:
# short indexed data 
# I just typed these out by hand for convenience.  
# Be careful with ordering!  Needs to match index sets above!
resources = [4, 2, 3, 1, 1]
unit_profits = [15, 9, 12, 6, 16.5, 13.5, 4.5] 

In [ ]:
## In this cell we're going to read in some csv data for the process_requirements, demand, and machine_down



### Converting to Dictionaries

The next step isn't strictly necessary and is a little wasteful in terms of inefficiency.  We could just use the numpy arrays directly.  But it's a good example of how to convert the data to dictionaries if you want to use them in the model.  And accessing data via dictionaries is sometimes more intuitive than accessing via array indexing.

In [ ]:
#create teh dictionaries here


In [ ]:
#let's also add one more dictionary that will be useful later, which is the available machine hours for each machine and month, accounting for machine down time


It's a good idea to print out your dictionaries (or parts of them) to make sure they look the way you intended.

Looks good! On to getting the full LP into Gurobi. These are the `Step 1`, `Step 2`, and `Step 3` in optimization model formulation that we know: decision variables, objective function, and constraints. Still though, we can be more systematic in our approach to this than we have been so far in class. For didactic purposes, let us actually add the constraints before the objective function. (*As long as they are all added, the order in which they are written does not matter.*)

In [ ]:
#create a gurobi model called factory
factory = Model('factory')

# VARIABLES



# CONSTRAINTS



# OBJECTIVE FUNCTION

# SOLVE

# optimize the model
factory.optimize()

In [ ]:
# PRINT RESULTS
